## Building Decision Tree From Scratch

### Import Libraries

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
%matplotlib inline

### Read Data

In [0]:
df = pd.read_csv('/content/drive/My Drive/data_set/iris.csv')
df = df.rename(columns={"variety": "label"})

### Train Test Split

Write a function to split the dataset into traning and testing set.


In [0]:
def train_test_split(df, test_size):
 
    if isinstance(test_size, float):
        test_size = round(test_size * len(df))

    indices = df.index.tolist()
    test_indices = random.sample(population=indices, k=test_size)

    test_df = df.loc[test_indices]
    train_df = df.drop(test_indices)
    
    return train_df, test_df

In [0]:
#print to check if the train_test_split function is working or not
random.seed(0)
train_df, test_df = train_test_split(df, test_size=20)

### Helper Functions

The helper functions operate on a NumPy 2d-array. Therefore, let’s create a variable called “data” .

In [0]:
data = train_df.values

### Pure Node ?

Now if in our target labels there's only one class there should not be any split and we should make our classification with just one class.

In [0]:
def check_purity(data):
    label_column = data[:, -1]
    unique_classes = np.unique(label_column)

    if len(unique_classes) == 1:
        return True
    else:
        return False

###Classify

Now Make classification  after we have identified the pure node or the majority class present in that node.

In [0]:
def classify_data(data):
    label_column = data[:, -1]
    unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)

    index = counts_unique_classes.argmax()
    classification = unique_classes[index]
    
    return classification

## Potential Split 

Now if the node is not pure. We should find all the potential split from our features and choose the best split. Excluding the label column.

In [0]:
def get_potential_splits(data):
    potential_splits = {}
    _, n_columns = data.shape
    for column_index in range(n_columns - 1):        # excluding the last column which is the label
        potential_splits[column_index] = []
        values = data[:, column_index]
        unique_values = np.unique(values)

        for index in range(len(unique_values)):
            if index != 0:
                current_value = unique_values[index]
                previous_value = unique_values[index - 1]
                potential_split = (current_value + previous_value) / 2
                
                potential_splits[column_index].append(potential_split)
    
    return potential_splits

#### Split Data

Check for the different parameters for the split

In [0]:
def split_data(data, split_column, split_value):
    split_column_values = data[:, split_column]

    data_below = data[split_column_values <= split_value]
    data_above = data[split_column_values >  split_value]
    
    return data_below, data_above

## Lowest Overall Entropy

Calculate the entropy for every possible split and choose the one with the best split i.e low entropy.

In [0]:
def calculate_entropy(data):
    
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

In [0]:
def calculate_overall_entropy(data_below, data_above):
    
    n = len(data_below) + len(data_above)
    p_data_below = len(data_below) / n
    p_data_above = len(data_above) / n

    overall_entropy =  (p_data_below * calculate_entropy(data_below) 
                      + p_data_above * calculate_entropy(data_above))
    
    return overall_entropy

In [0]:
def determine_best_split(data, potential_splits):
        
    overall_entropy = 9999
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(data, split_column=column_index, split_value=value)
            current_overall_entropy = calculate_overall_entropy(data_below, data_above)

            if current_overall_entropy <= overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value

##Decsision Tree Algorithm

Representation of the decision tree

In [0]:

sub_tree = {"question": ["yes_answer", 
                         "no_answer"]}

In [0]:

example_tree = {"petal_width <= 0.8": ["Iris-setosa", 
                                      {"petal_width <= 1.65": [{"petal_length <= 4.9": ["Iris-versicolor", 
                                                                                        "Iris-virginica"]}, 
                                                                "Iris-virginica"]}]}

## Algorithm

In [0]:
def decision_tree_algorithm(df, counter=0, min_samples=2, max_depth=5):
    
    # data preparations
    if counter == 0:
        global COLUMN_HEADERS
        COLUMN_HEADERS = df.columns
        data = df.values
    else:
        data = df           
    
    
    # base cases
    if (check_purity(data)) or (len(data) < min_samples) or (counter == max_depth):
        classification = classify_data(data)
        
        return classification

    
    # recursive part
    else:    
        counter += 1

        # helper functions 
        potential_splits = get_potential_splits(data)
        split_column, split_value = determine_best_split(data, potential_splits) 
        data_below, data_above = split_data(data, split_column, split_value)  
        
        # instantiate sub-tree
        
        feature_name = COLUMN_HEADERS[split_column]
        question = "{} <= {}".format(feature_name, split_value)
        sub_tree = {question: []}
        
        # find answers (recursion)
        
        yes_answer = decision_tree_algorithm(data_below, counter, min_samples, max_depth)
        no_answer = decision_tree_algorithm(data_above, counter, min_samples, max_depth)
        
        # If the answers are the same, then there is no point in asking the qestion.
        # This could happen when the data is classified even though it is not pure
        # yet (min_samples or max_depth base cases).
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree

In [0]:
tree = decision_tree_algorithm(train_df, max_depth=3)
print(tree)

{'petal.width <= 0.8': ['Setosa', {'petal.width <= 1.65': [{'petal.length <= 4.95': ['Versicolor', 'Virginica']}, 'Virginica']}]}


###Classification



In [0]:

example = test_df.iloc[0]
example

sepal.length           5.1
sepal.width            2.5
petal.length             3
petal.width            1.1
label           Versicolor
Name: 98, dtype: object

In [0]:

def classify_example(example, tree):
    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split()

    # ask question
    if example[feature_name] <= float(value):
        answer = tree[question][0]
    else:
        answer = tree[question][1]

    # base case
    
    if not isinstance(answer, dict):
        return answer
    
    # recursive part
    
    else:
        residual_tree = answer
        return classify_example(example, residual_tree)

In [0]:
classify_example(example, tree)

'Versicolor'

## Calculate Accuracy

In [0]:
def calculate_accuracy(df, tree):

    df["classification"] = df.apply(classify_example, axis=1, args=(tree,))
    df["classification_correct"] = df["classification"] == df["label"]
    
    accuracy = df["classification_correct"].mean()
    
    return accuracy

In [0]:
accuracy = calculate_accuracy(test_df, tree)
accuracy

0.95